# 0. Setup Paths and Other Stuff

In [ ]:
import os
import shutil

CUSTOM_MODEL_NAME = 'my_ssd_mobilenet'
PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'

paths = {
    'APIMODEL_PATH': os.path.join('Tensorflow','models'),
    'SCRIPTS_PATH': os.path.join('Tensorflow','scripts'),

    'WORKSPACE_PATH': os.path.join('Tensorflow', 'workspace'),

    'ANNOTATION_PATH': os.path.join('Tensorflow', 'workspace','annotations'),
    'IMAGE_PATH': os.path.join('Tensorflow', 'workspace','images'),
    'PRETRAINED_MODEL_PATH': os.path.join('Tensorflow', 'workspace','pre-trained-models'),

    'MODEL_PATH': os.path.join('Tensorflow', 'workspace','models'),
    'CHECKPOINT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME),
    'OUTPUT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'export'),
    'TFLITE_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfliteexport')
 }

files = {
    'PIPELINE_CONFIG':os.path.join(paths['MODEL_PATH'],CUSTOM_MODEL_NAME, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], 'generate_tfrecord.py'),
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], 'label_map.pbtxt'),
    'TRAINING_SCRIPT': os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'model_main_tf2.py')
}

# 1. Load Trained Model From Checkpoint

In [ ]:
import os
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util

In [ ]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(paths['CHECKPOINT_PATH'], 'ckpt-16')).expect_partial()

@tf.function(reduce_retracing=True)
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

IMAGE_PATH ='Tensorflow/workspace/images/test/bgld_1.jpg'

# Load an image using OpenCV
# image = cv2.imread(r'C:\Users\jinji\OneDrive\Desktop\bachelor-thesis\source\object-detection\Tensorflow\workspace\images\test\heart.163d1ea3-817d-11ee-93e0-ead0fc871b59.jpg')
image = cv2.imread(IMAGE_PATH)
# Convert BGR to RGB
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Display the image using Matplotlib
plt.imshow(image_rgb)
plt.title('Your Image Title')
plt.axis('off')  # Turn off axis labels
plt.show()

# 2. Detect from an Image

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap(files['LABELMAP'])

In [ ]:
def perform_inference(image_path):
    # Preprocess input 
    img = cv2.imread(IMAGE_PATH)
    image_np = np.array(img)

    # Perform inference
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.uint8)
    detections = detect_fn(input_tensor)
    
    # Accessing detection outputs
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy() for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    return image_np, detections

In [ ]:
%matplotlib inline

# IMAGE_PATH = os.path.join(paths['IMAGE_PATH'], 'version1-2', 'test', 'Cars422.png')
IMAGE_PATH = os.path.join(paths['IMAGE_PATH'], 'test', 'sonder_8.jpg')
# IMAGE_PATH = os.path.join(paths['IMAGE_PATH'], 'test', 'VideoCapture_20231114-110614.jpg')

image_np, detections = perform_inference(IMAGE_PATH)

label_id_offset = 1
image_np_with_detections = image_np.copy()

# Visualize the results
viz_utils.visualize_boxes_and_labels_on_image_array(
    image_np_with_detections,
    detections['detection_boxes'],
    detections['detection_classes'] + label_id_offset,
    detections['detection_scores'],
    category_index,
    use_normalized_coordinates=True,
    max_boxes_to_draw=4,
    min_score_thresh=0.5,
    agnostic_mode=False)

plt.imshow(cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()

# 3. Real Time Detections from your Webcam

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# cap = cv2.VideoCapture(0)
cap = cv2.VideoCapture(os.path.join(paths['IMAGE_PATH'], 'Autos', '20231106_222316.mp4'))

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

while cap.isOpened():
    ret, frame = cap.read()
    image_np = np.array(frame)
    
    if not ret:
        print("End of video.")
        break

    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0).astype(np.uint8))
    detections = detect_fn(input_tensor)

    # Remove extra dim from array
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections   
    
    # Extract information from the detections
    boxes = detections['detection_boxes']
    classes = detections['detection_classes'].astype(int)
    scores = detections['detection_scores']

    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                boxes,
                classes,
                scores,
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=5,
                min_score_thresh=.2,
                agnostic_mode=False)

    cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (360, 640)))

    if cv2.waitKey(10) & 0xFF == ord('q'):
        print("Exit.")
        break
    
cap.release()
cv2.destroyAllWindows()

NameError: name 'cv2' is not defined